In [1]:
import os
import random
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision

from torch.utils.data import Dataset, DataLoader, BatchSampler, random_split
from torchvision import transforms
from PIL import Image

In [2]:
super_prob_cut = 0.7 #probability cut-off for test image belonging to highest probability superclass from training data (if lower, classified as novel)
sub_prob_cut = 0.4 #probability cut-off for test image belonging to highest probability subclass from training data (if lower, classified as novel)


In [3]:
from torchvision import models
backbone = models.resnet18(weights='IMAGENET1K_V1')

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 56.3MB/s]


In [4]:
backbone

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [5]:
from google.colab import drive
drive.mount('/content/drive')
%cd drive/MyDrive/NNDL_Project

Mounted at /content/drive
/content/drive/MyDrive/NNDL_Project


In [6]:
%%capture
! unzip -o train_shuffle.zip

In [7]:
%%capture
! unzip -o test_shuffle.zip

In [8]:
# Create Dataset class for multilabel classification
class MultiClassImageDataset(Dataset):
    def __init__(self, ann_df, super_map_df, sub_map_df, img_dir, transform=None):
        self.ann_df = ann_df
        self.super_map_df = super_map_df
        self.sub_map_df = sub_map_df
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.ann_df)

    def __getitem__(self, idx):
        img_name = self.ann_df['image'][idx]
        img_path = os.path.join(self.img_dir, img_name)
        image = Image.open(img_path).convert('RGB')

        super_idx = self.ann_df['superclass_index'][idx]
        super_label = self.super_map_df['class'][super_idx]

        sub_idx = self.ann_df['subclass_index'][idx]
        sub_label = self.sub_map_df['class'][sub_idx]

        if self.transform:
            image = self.transform(image)

        return image, super_idx, super_label, sub_idx, sub_label

class MultiClassImageTestDataset(Dataset):
    def __init__(self, super_map_df, sub_map_df, img_dir, transform=None):
        self.super_map_df = super_map_df
        self.sub_map_df = sub_map_df
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self): # Count files in img_dir
        return len([fname for fname in os.listdir(self.img_dir)])

    def __getitem__(self, idx):
        img_name = str(idx) + '.jpg'
        img_path = os.path.join(self.img_dir, img_name)
        image = Image.open(img_path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        return image, img_name

In [9]:
train_ann_df = pd.read_csv('train_data.csv')
super_map_df = pd.read_csv('superclass_mapping.csv')
sub_map_df = pd.read_csv('subclass_mapping.csv')

train_img_dir = 'train_shuffle'
test_img_dir = 'test_shuffle'

image_preprocessing = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=(0), std=(1)),
])

# Create train and val split
train_dataset = MultiClassImageDataset(train_ann_df, super_map_df, sub_map_df, train_img_dir, transform=image_preprocessing)
train_dataset, val_dataset = random_split(train_dataset, [0.9, 0.1])

# Create test dataset
test_dataset = MultiClassImageTestDataset(super_map_df, sub_map_df, test_img_dir, transform=image_preprocessing)

# Create dataloaders
batch_size = 64
train_loader = DataLoader(train_dataset,
                          batch_size=batch_size,
                          shuffle=True, drop_last = True)

val_loader = DataLoader(val_dataset,
                        batch_size=batch_size,
                        shuffle=True, drop_last = True)

test_loader = DataLoader(test_dataset,
                         batch_size=1,
                         shuffle=False)

In [10]:
# Simple CNN
class CNN(nn.Module):
    def __init__(self, backbone, finetune=False):
        super().__init__()

        self.backbone = backbone
        num_ftrs = self.backbone.fc.in_features
        print('num_ftrs: ', num_ftrs)
        self.backbone.fc = torch.nn.Identity()
        print(self.backbone)

        if not finetune:
          for param in self.backbone.parameters():
            param.requires_grad = False
        else:
          for i, layer in enumerate(self.backbone.children()):
            if i < 6: #just train last of the main convolutional layers
              for param in layer.parameters():
                param.requires_grad = False
            else:
              for param in layer.parameters():
                param.requires_grad = True


        # Alternatively, it can be generalized to ``nn.Linear(num_ftrs, len(class_names))``.
        self.fca= nn.Linear(num_ftrs, 4)
        self.fcb= nn.Linear(num_ftrs, 88)


    def forward(self, x):
        x_new = self.backbone(x)
        super_out = self.fca(x_new)
        sub_out = self.fcb(x_new)
        return super_out, sub_out

class Trainer():
    def __init__(self, model, criterion, optimizer, train_loader, val_loader, test_loader=None, device='cuda'):
        self.model = model
        self.criterion = criterion
        self.optimizer = optimizer
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.test_loader = test_loader
        self.device = device

    def train_epoch(self):
        running_loss = 0.0
        device = self.device
        for i, data in enumerate(self.train_loader):
            inputs, super_labels, sub_labels = data[0].to(device), data[1].to(device), data[3].to(device)

            self.optimizer.zero_grad()
            super_outputs, sub_outputs = self.model(inputs)
            loss = self.criterion(super_outputs, super_labels) + self.criterion(sub_outputs, sub_labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f'Training loss: {running_loss/i:.3f}')

    def validate_epoch(self):
        super_correct = 0
        sub_correct = 0
        total = 0
        running_loss = 0.0
        device = self.device
        with torch.no_grad():
            for i, data in enumerate(self.val_loader):
                inputs, super_labels, sub_labels = data[0].to(device), data[1].to(device), data[3].to(device)

                super_outputs, sub_outputs = self.model(inputs)
                loss = self.criterion(super_outputs, super_labels) + self.criterion(sub_outputs, sub_labels)
                _, super_predicted = torch.max(super_outputs.data, 1)
                _, sub_predicted = torch.max(sub_outputs.data, 1)

                total += super_labels.size(0)
                super_correct += (super_predicted == super_labels).sum().item()
                sub_correct += (sub_predicted == sub_labels).sum().item()
                running_loss += loss.item()

        print(f'Validation loss: {running_loss/i:.3f}')
        print(f'Validation superclass acc: {100 * super_correct / total:.2f} %')
        print(f'Validation subclass acc: {100 * sub_correct / total:.2f} %')

    def test(self, save_to_csv=False, return_predictions=False):
        if not self.test_loader:
            raise NotImplementedError('test_loader not specified')

        # Evaluate on test set, in this simple demo no special care is taken for novel/unseen classes
        test_predictions = {'image': [], 'superclass_index': [], 'subclass_index': []}
        with torch.no_grad():
            for i, data in enumerate(self.test_loader):
                inputs, img_name = data[0].to(device), data[1]

                super_outputs, sub_outputs = self.model(inputs)
                if i < 50: #get an idea of probabilities
                  print('softmax super: ', torch.max(torch.nn.functional.softmax(super_outputs.data, dim=1)))
                  print('softmax super beats cut: ', torch.max(torch.nn.functional.softmax(super_outputs.data, dim=1)) > super_prob_cut)
                  print('softmax sub: ', torch.max(torch.nn.functional.softmax(sub_outputs.data, dim=1)))
                  print('softmax sub beats cut: ', torch.max(torch.nn.functional.softmax(sub_outputs.data, dim=1)) > sub_prob_cut)
                  print('')
                  print('')

                if torch.max(torch.nn.functional.softmax(super_outputs.data, dim=1)) < super_prob_cut:
                  super_predicted = 3 #novel
                  test_predictions['superclass_index'].append(super_predicted)
                  sub_predicted = 87 #novel
                  test_predictions['subclass_index'].append(sub_predicted)
                else:
                  _, super_predicted = torch.max(super_outputs.data, 1)
                  test_predictions['superclass_index'].append(super_predicted.item())
                  if torch.max(torch.nn.functional.softmax(sub_outputs.data, dim=1)) < sub_prob_cut:
                    sub_predicted = 87 #novel
                    test_predictions['subclass_index'].append(sub_predicted)
                  else:
                    _, sub_predicted = torch.max(sub_outputs.data, 1)
                    test_predictions['subclass_index'].append(sub_predicted.item())

                test_predictions['image'].append(img_name[0])

        test_predictions = pd.DataFrame(data=test_predictions)

        if save_to_csv:
            test_predictions.to_csv('probabilistic_superandsub_test_predictions.csv', index=False)

        if return_predictions:
            return test_predictions

In [11]:
def fine_tune(backbone):
  for i, layer in enumerate(backbone.children()):
    if i < 6: #just train last of the main convolutional layers
      for param in layer.parameters():
        param.requires_grad = False
    else:
      for param in layer.parameters():
        param.requires_grad = True
  return

In [12]:
# Init model and trainer
device = 'cuda'
backbone = models.resnet18(weights='IMAGENET1K_V1')
finetune = False
model = CNN(backbone, finetune=finetune).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)
trainer = Trainer(model, criterion, optimizer, train_loader, val_loader, test_loader)

num_ftrs:  512
ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(i

In [13]:
# Training loop
print('Trainable parameters: ', sum(p.numel() for p in model.parameters() if p.requires_grad))
print('Total parameters: ', sum(p.numel() for p in model.parameters()))
print('----------------------')
print('')
for epoch in range(20):
    print(f'Epoch {epoch+1}')
    trainer.train_epoch()
    trainer.validate_epoch()
    print('')
    if epoch == 10:
      fine_tune(trainer.model.backbone)
      print('')
      print('Trainable parameters: ', sum(p.numel() for p in trainer.model.parameters() if p.requires_grad))
      print('Total parameters: ', sum(p.numel() for p in trainer.model.parameters()))
      print('----------------------')
      print('')
      trainer = Trainer(trainer.model, trainer.criterion, trainer.optimizer, trainer.train_loader, trainer.val_loader, trainer.test_loader)


print('Finished Training')

Trainable parameters:  47196
Total parameters:  11223708
----------------------

Epoch 1
Training loss: 4.963
Validation loss: 4.731
Validation superclass acc: 72.22 %
Validation subclass acc: 20.31 %

Epoch 2
Training loss: 3.745
Validation loss: 4.209
Validation superclass acc: 74.65 %
Validation subclass acc: 23.26 %

Epoch 3
Training loss: 3.276
Validation loss: 4.049
Validation superclass acc: 73.09 %
Validation subclass acc: 27.43 %

Epoch 4
Training loss: 3.032
Validation loss: 4.049
Validation superclass acc: 76.74 %
Validation subclass acc: 27.78 %

Epoch 5
Training loss: 2.888
Validation loss: 3.915
Validation superclass acc: 76.56 %
Validation subclass acc: 31.42 %

Epoch 6
Training loss: 2.754
Validation loss: 3.918
Validation superclass acc: 75.35 %
Validation subclass acc: 28.65 %

Epoch 7
Training loss: 2.642
Validation loss: 3.933
Validation superclass acc: 78.12 %
Validation subclass acc: 29.17 %

Epoch 8
Training loss: 2.531
Validation loss: 3.976
Validation superclas

In [14]:
trainer.model.eval()
predictions = trainer.test(save_to_csv=True, return_predictions=True)

'''
This simple baseline scores the following test accuracy

Superclass Accuracy
Overall: 43.83 %
Seen: 61.11 %
Unseen: 0.00 %

Subclass Accuracy
Overall: 2.03 %
Seen: 9.56 %
Unseen: 0.00 %
'''

softmax super:  tensor(1.0000, device='cuda:0')
softmax super beats cut:  tensor(True, device='cuda:0')
softmax sub:  tensor(0.5197, device='cuda:0')
softmax sub beats cut:  tensor(True, device='cuda:0')


softmax super:  tensor(0.9999, device='cuda:0')
softmax super beats cut:  tensor(True, device='cuda:0')
softmax sub:  tensor(0.8284, device='cuda:0')
softmax sub beats cut:  tensor(True, device='cuda:0')


softmax super:  tensor(0.9892, device='cuda:0')
softmax super beats cut:  tensor(True, device='cuda:0')
softmax sub:  tensor(0.8932, device='cuda:0')
softmax sub beats cut:  tensor(True, device='cuda:0')


softmax super:  tensor(0.9437, device='cuda:0')
softmax super beats cut:  tensor(True, device='cuda:0')
softmax sub:  tensor(0.9763, device='cuda:0')
softmax sub beats cut:  tensor(True, device='cuda:0')


softmax super:  tensor(0.9999, device='cuda:0')
softmax super beats cut:  tensor(True, device='cuda:0')
softmax sub:  tensor(0.9693, device='cuda:0')
softmax sub beats cut:  te

'\nThis simple baseline scores the following test accuracy\n\nSuperclass Accuracy\nOverall: 43.83 %\nSeen: 61.11 %\nUnseen: 0.00 %\n\nSubclass Accuracy\nOverall: 2.03 %\nSeen: 9.56 %\nUnseen: 0.00 %\n'

In [15]:
predictions

,image,superclass_index,subclass_index
0,0.jpg,1,46
1,1.jpg,0,86
2,2.jpg,2,29
3,3.jpg,2,43
4,4.jpg,1,49
...,...,...,...
12372,12372.jpg,0,11
12373,12373.jpg,2,15
12374,12374.jpg,2,44
12375,12375.jpg,0,56


In [16]:
predictions[predictions['superclass_index']==0]

,image,superclass_index,subclass_index
1,1.jpg,0,86
6,6.jpg,0,24
8,8.jpg,0,56
11,11.jpg,0,87
12,12.jpg,0,82
...,...,...,...
12363,12363.jpg,0,5
12367,12367.jpg,0,27
12369,12369.jpg,0,87
12372,12372.jpg,0,11


In [17]:
predictions['superclass_index'].value_counts()

0    4776
1    3804
2    2746
3    1051
Name: superclass_index, dtype: int64

In [18]:
predictions['subclass_index'].value_counts()

87    2412
35     303
44     302
46     290
11     290
      ... 
67      26
50      25
75      25
69      25
63      24
Name: subclass_index, Length: 88, dtype: int64